In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [4]:
conf = SparkConf()
conf.set("spark.hadoop.fs.s3a.impl",
                 "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "123456789")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider',
            'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set(
    "spark.jars.packages",
    "io.delta:delta-core_2.12:1.0.0,"
    "org.apache.hadoop:hadoop-aws:3.1.1,"
    "com.amazonaws:aws-java-sdk:1.11.271,"
    "com.amazonaws:aws-java-sdk-bundle:1.11.271,"
    "software.amazon.awssdk:url-connection-client:2.15.40",
)
conf.set("spark.sql.extensions",
            "io.delta.sql.DeltaSparkSessionExtension")
conf.set(
    "spark.sql.catalog.spark_catalog",
    "org.apache.spark.sql.delta.catalog.DeltaCatalog",
)
conf.set("spark.databricks.delta.merge.repartitionBeforeWrite.enabled","true")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .master("local[*]")
    .getOrCreate()
)

In [5]:
base_path = "s3a://datalake/brozen/"

#### Delta table history

#### Get history of table

In [30]:
def get_full_history_table(table_name: str):
    deltaTable = DeltaTable.forPath(spark, base_path + table_name)

    fullHistoryDF = deltaTable.history()    # get the full history of the table

    return fullHistoryDF

df_history = get_full_history_table("cdc.myshop.orders")
df_history.show(truncate=False)

+-------+-------------------+------+--------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|version|timestamp          |userId|userName|operation|operationParameters                                                                                                                                                                                                                                       

In [31]:
df_history.select("version", "timestamp", "operation").show(truncate=False)

+-------+-------------------+---------+
|version|timestamp          |operation|
+-------+-------------------+---------+
|14     |2023-04-22 23:19:18|MERGE    |
|13     |2023-04-22 23:18:17|MERGE    |
|12     |2023-04-22 23:17:15|MERGE    |
|11     |2023-04-22 23:16:18|MERGE    |
|10     |2023-04-22 23:15:21|MERGE    |
|9      |2023-04-22 23:14:20|MERGE    |
|8      |2023-04-22 23:13:17|MERGE    |
|7      |2023-04-22 23:12:25|MERGE    |
|6      |2023-04-22 23:11:16|MERGE    |
|5      |2023-04-22 23:10:19|MERGE    |
|4      |2023-04-22 23:09:19|MERGE    |
|3      |2023-04-22 23:08:20|MERGE    |
|2      |2023-04-22 23:07:27|MERGE    |
|1      |2023-04-22 23:06:18|MERGE    |
|0      |2023-04-22 22:23:29|WRITE    |
+-------+-------------------+---------+



#### Time travel Query with version of table

In [27]:
def time_travel_query_version(table_name: str, version: str):
    df_res = (
        spark.read
        .format("delta")
        .option("versionAsOf", version)
        .load(base_path + table_name)
    )
    return df_res

df = time_travel_query_version("cdc.myshop.orders", "14")
df.show(truncate=False)
df.count()

+---+-------------+---+-------+-----------+---------+----------+---------+----+-----+---+
|op |ts_ms        |id |user_id|payment    |status_id|created_at|before_id|year|month|day|
+---+-------------+---+-------+-----------+---------+----------+---------+----+-----+---+
|r  |1682161423510|112|6528   |instalment |2        |1660146044|112      |2022|8    |10 |
|r  |1682161423510|123|7665   |instalment |2        |1660146044|123      |2022|8    |10 |
|r  |1682161423508|18 |70     |credit_card|3        |1657122044|18       |2022|7    |6  |
|r  |1682161423511|136|2793   |credit_card|3        |1657122044|136      |2022|7    |6  |
|r  |1682161423508|17 |185    |cash       |1        |1664552444|17       |2022|9    |30 |
|r  |1682161423510|84 |742    |cash       |2        |1664552444|84       |2022|9    |30 |
|r  |1682161423511|140|6254   |instalment |4        |1664552444|140      |2022|9    |30 |
|r  |1682161423510|100|5339   |credit_card|3        |1675179644|100      |2023|1    |31 |
|r  |16821

240

#### Time travel query with timestamp

In [ ]:
def time_travel_query_timestamp(table_name: str, timestamp: str):
    df_res = (
        spark.read
        .format("delta")
        .option("timestampAsOf", timestamp)
        .load(base_path + table_name)
    )
    return df_res

df = time_travel_query_timestamp("cdc.myshop.orders", "2023-04-16")
df.show(truncate=False)
df.count()